# 3_data_nrw_names_fixed

In this script I'm going to fix all city names from the NRW database

## Why is that necessary?
The cities have been written in an uneven way, which means there are typos, plz codes and it might have two different cities with the same name in this database. We are making sure everything is clear.

### Conclusions:

We have found great inconsistency in city data.
We wanted to see which municipalities uploaded their data in the vergabe.nrw system, so we looked at the variable ```result_buyer_town```. 
The first challenge was to verify the city name. Since the public servant needs to right the city name himself, the data was full of typos, non-oficial names (i.e. Gronau and Gronau (West.) ) and we have found even towns that don't belong to NRW (Berlin, Ösnabruck and Dillenburg). We did not investigated why those cities were present with the data, but we have the hypothesis that it could either had to do with purchases made by the national government for services/products that would be directed to NRW (in case of Berlin) or Tenders that were also published in NRW so companies from the state could participate in it.


There database has 215 of the 396 cities, which means that 181 cities are missing (list of the available cities and missing cities are in the end of the document). An amount of 605 entries related to 22 cities needed to be corrected either due to typos, non-oficial name or because they were related to neighbordoods (Stadtteil). Besides, 11 inputs refer to cities outside NRW (Berlin = 6, Ösnabruck = 4 and Dillenburg = 1).

### Code:

In [49]:
import pandas as pd
import matplotlib 
import numpy as np
import janitor

In [50]:
# This does not belong to the analysis, I'm just creating a function to help me lates:

def glimpse(df, maxvals=10, maxlen=110):
    print('Shape: ', df.shape)
    
    def pad(y):
        max_len = max([len(x) for x in y])
        return [x.ljust(max_len) for x in y]
    
    # Column Name
    toprnt = pad(df.columns.tolist())
    
    # Column Type
    toprnt = pad([toprnt[i] + ' ' + str(df.iloc[:,i].dtype) for i in range(df.shape[1])])
    
    # Num NAs
    num_nas = [df.iloc[:,i].isnull().sum() for i in range(df.shape[1])]
    num_nas_ratio = [int(round(x*100/df.shape[0])) for x in num_nas]
    num_nas_str = [str(x) + ' (' + str(y) + '%)' for x,y in zip(num_nas, num_nas_ratio)]
    max_len = max([len(x) for x in num_nas_str])
    num_nas_str = [x.rjust(max_len) for x in num_nas_str]
    toprnt = [x + ' ' + y + ' NAs' for x,y in zip(toprnt, num_nas_str)]
    
    # Separator
    toprnt = [x + ' : ' for x in toprnt]
    
    # Values
    toprnt = [toprnt[i] + ', '.join([str(y) for y in df.iloc[:min([maxvals,df.shape[0]]), i]]) for i in range(df.shape[1])]
    
    # Trim to maxlen
    toprnt = [x[:min(maxlen, len(x))] for x in toprnt]
    
    for x in toprnt:
        print(x)
        

####sec function:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [51]:
#import
data_nrw_valid_ids = pd.read_csv("data_nrw_valid_ids.csv", index_col='_id', low_memory=False)

### 1. Create a variable with the clean name:

In [52]:
# dicionário de caracteres que serão substituídos:
caracteres = {ord('ä'):'ae', ord('ü'):'ue', ord('ö'):'oe', ord('ß'):'ss'}

data_nrw_clean_cities = data_nrw_valid_ids

data_nrw_clean_cities['controle'] = (
    data_nrw_clean_cities['result_buyer_town'].str.lower()
    .str.translate(caracteres)
    .str.strip()
    .replace({'\s+': ' '}, regex = True)
)

data_nrw_clean_cities['cleaned_buyer_town'] = (
    data_nrw_clean_cities['result_buyer_town'].str.lower()
    .str.translate(caracteres)
    .str.strip()
    .replace({'schleiden / 53937' : 'schleiden', 
              '59192' :  'bergkamen', 
              '50259 pulheim' : 'pulheim',
              'stadt pulheim' : 'pulheim',
              'euv stadtbetrieb castrop rauxel aoer' : 'castrop-rauxel',
              'euv stadtbetrieb castrop-rauxel aoer' : 'castrop-rauxel',
              'schleiden   53937': 'schleiden',
              'delbrueck-westenholz': 'delbrueck',
              'gronau$': 'gronau (westf.)',
              'bergisch-gladbach':  'bergisch gladbach',
              'bergsich gladbach': 'bergisch gladbach',
              'bruehl-ost' : 'bruehl',
              'bielefdeld' : 'bielefeld',
              'froendenberg$':'froendenberg/ruhr',
              'nettersheim-zingsheim': 'nettersheim',
              'swisttal-miel': 'swisttal',
              'troisdorf-sieglar': 'troisdorf',
              'warburg-scherfede':'warburg',
              'hennef' :'hennef (sieg)',
              'langenfeld': 'langenfeld (rhld.)',
              'leichlingen' : 'leichlingen (rhld.)',
              'stolbeerg' : 'stolberg (rhld.)',
              'stolberg' : 'stolberg (rhld.)',
              'sundern' : 'sundern (sauerland)',
              'uebach palenberg' : 'uebach-palenberg',
              'schhwelm' : 'schwelm',      
              '\s+': ' '}, regex = True)
)      


data_nrw_clean_cities['mudou'] = np.where(data_nrw_clean_cities['cleaned_buyer_town'] == data_nrw_clean_cities['controle'], 
                                          0, 1)
not_nrw = ['osnabrueck','dillenburg','berlin']

#saving who is not in nrw:
out_nrw = data_nrw_clean_cities[data_nrw_clean_cities.cleaned_buyer_town.isin(not_nrw)]

#filtering
data_nrw_clean_cities = data_nrw_clean_cities[~data_nrw_clean_cities.cleaned_buyer_town.isin(not_nrw)]

In [53]:
print('Number of cities before:', len(data_nrw_valid_ids.result_buyer_town.unique()), '\nNumber of cities after:',
     len(data_nrw_clean_cities.cleaned_buyer_town.unique()), '\n\nAn amount of', 
      (len(data_nrw_valid_ids.result_buyer_town.unique())-len(data_nrw_clean_cities.cleaned_buyer_town.unique())),
      'cities needed to be corrected. Either they were neighborhoods ' )

Number of cities before: 237 
Number of cities after: 215 

An amount of 22 cities needed to be corrected. Either they were neighborhoods 


In [54]:
print('In sum we modified a total of',data_nrw_clean_cities.mudou.sum(), 'entries')

In sum we modified a total of 605 entries


Now we will see the oficial cities that are not present in our database:

In [55]:
# just checking how many inputs doesn't belong to nrw:

print('Inputs where buyer_town was not in NRW \n',out_nrw.groupby(by = 'cleaned_buyer_town').agg({'cleaned_buyer_town':['count']}))

Inputs where buyer_town was not in NRW 
                    cleaned_buyer_town
                                count
cleaned_buyer_town                   
berlin                              6
dillenburg                          1
osnabrueck                          4


In [56]:
#Saving the cities to double-check:

only_cities = data_nrw_clean_cities.reset_index()
only_cities = only_cities[["cleaned_buyer_town"]]
only_cities = only_cities.drop_duplicates()

only_cities.to_csv('only_cities.csv', index = False)

In [57]:
#openning oficial names from  cities in NRW:
stadt_nrw_oficial = pd.read_csv("georef-germany-gemeinde.csv", sep=';').clean_names()

stadt_nrw_oficial['cleaned_buyer_town'] = (
    stadt_nrw_oficial['gemeinde_name_short_'].str.lower()
    .str.translate(caracteres)
    .str.strip()
)


#looking if all the cities are here:
cities_with_vergabe = data_nrw_clean_cities.cleaned_buyer_town.unique()
oficial_cities = stadt_nrw_oficial.cleaned_buyer_town.unique()

missing_cities = set(oficial_cities) - set(cities_with_vergabe)

print('there are',len(missing_cities), 'cities missing')

there are 181 cities missing


Now we are going to comparre with an official Database of german cities and geopoints

In [58]:
#finally saving the object with cleaned cities:

data_nrw_clean_cities = data_nrw_clean_cities.drop(['mudou', 'controle', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [59]:
#saving:

data_nrw_clean_cities.to_csv('3_data_nrw_clean_cities.csv')

### Anex:

### Cities contained in the database:

In [60]:
sorted(data_nrw_clean_cities.cleaned_buyer_town.unique())

['aachen',
 'ahaus',
 'ahlen',
 'alsdorf',
 'arnsberg',
 'ascheberg',
 'attendorn',
 'bad driburg',
 'bad honnef',
 'bad muenstereifel',
 'baesweiler',
 'balve',
 'beckum',
 'bergheim',
 'bergisch gladbach',
 'bergkamen',
 'bergneustadt',
 'bielefeld',
 'billerbeck',
 'blankenheim',
 'bocholt',
 'bochum',
 'boenen',
 'bonn',
 'borgentreich',
 'borken',
 'bornheim',
 'bottrop',
 'brilon',
 'bruehl',
 'buende',
 'burscheid',
 'castrop-rauxel',
 'coesfeld',
 'dahlem',
 'datteln',
 'delbrueck',
 'detmold',
 'dorsten',
 'dortmund',
 'dueren',
 'duesseldorf',
 'duisburg',
 'eitorf',
 'emmerich am rhein',
 'emsdetten',
 'ennigerloh',
 'erftstadt',
 'erkelenz',
 'erkrath',
 'eschweiler',
 'essen',
 'euskirchen',
 'everswinkel',
 'frechen',
 'froendenberg/ruhr',
 'gangelt',
 'geilenkirchen',
 'geldern',
 'gelsenkirchen',
 'greven',
 'gronau (westf.)',
 'gummersbach',
 'hagen',
 'haltern am see',
 'halver',
 'hamm',
 'hattingen',
 'heiden',
 'heiligenhaus',
 'heimbach',
 'heinsberg',
 'hemer',
 

### Cities missing in the database:

In [61]:
sorted(missing_cities)

['aldenhoven',
 'alfter',
 'alpen',
 'altena',
 'altenbeken',
 'altenberge',
 'anroechte',
 'augustdorf',
 'bad berleburg',
 'bad laasphe',
 'bad lippspringe',
 'bad oeynhausen',
 'bad salzuflen',
 'bad sassendorf',
 'bad wuennenberg',
 'barntrup',
 'bedburg',
 'bedburg-hau',
 'beelen',
 'bestwig',
 'beverungen',
 'blomberg',
 'borchen',
 'borgholzhausen',
 'brakel',
 'breckerfeld',
 'brueggen',
 'bueren',
 'burbach',
 'dinslaken',
 'doerentrup',
 'dormagen',
 'drensteinfurt',
 'drolshagen',
 'duelmen',
 'elsdorf',
 'engelskirchen',
 'enger',
 'ennepetal',
 'ense',
 'erndtebrueck',
 'erwitte',
 'eslohe (sauerland)',
 'espelkamp',
 'extertal',
 'finnentrop',
 'freudenberg',
 'gescher',
 'geseke',
 'gevelsberg',
 'gladbeck',
 'goch',
 'grefrath',
 'grevenbroich',
 'guetersloh',
 'haan',
 'halle (westf.)',
 'hallenberg',
 'hamminkeln',
 'harsewinkel',
 'havixbeck',
 'heek',
 'hellenthal',
 'herdecke',
 'herten',
 'herzebrock-clarholz',
 'hiddenhausen',
 'hille',
 'hoexter',
 'hopsten',
 '